## Load data

In [5]:
df = spark.read.option("multiline", "true").json("Files/news/us-latest-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/news/us-latest-news.json".
df = df.select("articles")
display(df)

StatementMeta(, 3a7a3a36-e001-4c83-8e47-a6e448a1573c, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8494ed64-19df-4e52-81ef-c6af33615eff)

## Exlode dataframe 

In [6]:
from pyspark.sql.functions import explode, col
from pyspark.sql.types import StructField, StructType, StringType

# Correct explode operation
exploded_df = df.select(explode(col("articles")).alias('article'))

# Extract columns directly from the exploded DataFrame
clean_df = exploded_df.select(
    col("article.title").alias("title"),
    col("article.author").alias("author"),
    col("article.description").alias("description"),
    col("article.publishedAt").alias("datePublished"),
    col("article.url").alias("url"),
    col("article.source.name").alias("source")
)

# Optional: Date formatting
from pyspark.sql.functions import to_date, date_format
clean_df = clean_df.withColumn(
    'datePublished', 
    date_format(to_date(col('datePublished')), "dd-MMM-yyyy")
)

# Display the DataFrame
display(clean_df)

StatementMeta(, 3a7a3a36-e001-4c83-8e47-a6e448a1573c, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f6f27d88-4710-4c1b-b87a-afddcdabf2c6)

## Save data in lakehouse

In [9]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = "news_lakehouse_1.tbl_latest_us_news"
    clean_df.write.delta('delta').saveAsTable(table_name)
except Exception as e:
    print("Table already exists")
    clean_df.createOrReplaceTempView("vw_clean_df")
    spark.sql(f"""
            MERGE INTO {table_name} target_table
            USING vw_clean_df source_view
            ON source_view.url = target_table.url
            WHEN MATCHED AND 
            source_view.title <> target_table.title OR 
            source_view.author <> target_table.author OR
            source_view.description <> target_table.description OR
            source_view.source <> target_table.source OR
            source_view.datePublished <> target_table.datePublished 
            THEN UPDATE SET *
            WHEN NOT MATCHED THEN INSERT *
    """)


StatementMeta(, 3a7a3a36-e001-4c83-8e47-a6e448a1573c, 11, Finished, Available, Finished)

Table already exists
